In [44]:
# All necessary libraries must be put here
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import mplfinance as mpf
import plotly.graph_objects as go
import pandas as pd
import cv2
import matplotlib.image as mpimg
import os
import warnings

In [16]:
# Read the CRSP Data and make it become a dataframe
crsp_data_1993_2019 = pd.read_csv("/Users/namnguyen/Desktop/Research/Don/Imaging_Price_Trends_Code/crsp_data_20240719_1.csv")

# Check the data
crsp_data_1993_2019.head(5)

/var/folders/pl/z1lfm5xs7299k23shpwd6ngh0000gp/T/ipykernel_94651/4244800754.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  crsp_data_1993_2019 = pd.read_csv("/Users/namnguyen/Desktop/Research/Don/Imaging_Price_Trends_Code/crsp_data_20240719_1.csv")


,PERMNO,HdrCUSIP,Ticker,PERMCO,DlyCalDt,DlyVol,DlyClose,DlyLow,DlyHigh,DlyOpen
0,10001,36720410,GFGC,7953,1993-01-04,150.0,14.5,14.50,14.5,14.50
1,10001,36720410,GFGC,7953,1993-01-05,0.0,NaN,NaN,NaN,NaN
2,10001,36720410,GFGC,7953,1993-01-06,0.0,NaN,NaN,NaN,NaN
3,10001,36720410,GFGC,7953,1993-01-07,228.0,14.5,14.50,14.5,14.50
4,10001,36720410,GFGC,7953,1993-01-08,1375.0,14.5,14.25,14.5,14.25


In [17]:
# At this part, we focus first on training sample
# Training data are taken from 1993 to 2000. 

crsp_data_1993_2019["DlyCalDt"] = pd.to_datetime(crsp_data_1993_2019["DlyCalDt"])

# Filter dates from 1993 to 2000
start_date = "1993-01-01"
end_date = "2000-12-31" 
crsp_data_1993_2000 = crsp_data_1993_2019[(crsp_data_1993_2019["DlyCalDt"] >= start_date) & (crsp_data_1993_2019["DlyCalDt"] <= end_date)]

# Check the last 5 values
print(crsp_data_1993_2000.tail(5))

# Check the shape of the crsp_data_1993_2000
print(crsp_data_1993_2000.shape) # (17223740, 10)

# Check the shape of the crsp_data_1993_2019
print(crsp_data_1993_2019.shape) # (51131155, 10)

          PERMNO  HdrCUSIP Ticker  PERMCO   DlyCalDt   DlyVol  DlyClose  \
50923819   93316  98960110   ZING    7469 2000-03-13  10685.0   15.3750   
50923820   93316  98960110   ZING    7469 2000-03-14   1800.0   15.0625   
50923821   93316  98960110   ZING    7469 2000-03-15   2100.0   15.0625   
50923822   93316  98960110   ZING    7469 2000-03-16    700.0   15.3750   
50923823   93316  98960110    NaN    7469 2000-03-17      NaN       NaN   

           DlyLow  DlyHigh  DlyOpen  
50923819  15.1250   15.375  15.2500  
50923820  15.0625   15.375  15.3750  
50923821  15.0625   15.375  15.0625  
50923822  15.0625   15.375  15.0625  
50923823      NaN      NaN      NaN  
(17223740, 10)
(51131155, 10)


In [18]:
# Replace the NaN values with 0
crsp_data_1993_2000 = crsp_data_1993_2000.dropna()

# Check the shape again
print(crsp_data_1993_2000.shape)

(15555109, 10)


In [19]:
# Drop unused columns in the dataframe
crsp_data_1993_2000 = crsp_data_1993_2000.drop(["HdrCUSIP", "Ticker", "PERMCO"], axis = 1)

# Check the shape again
print(crsp_data_1993_2000.shape) # (15555109, 7)

(15555109, 7)


In [46]:
# Function to calculate 5-day log returns
def calculate_5_day_log_returns(group):
    # Create return for each day
    group["Return"] = group["DlyClose"].pct_change()

    # Shift it back 1 column
    group["Return"] = group["Return"].shift(-1)
    group["Return"].fillna(method = "ffill", inplace = True)
    
    # Calculate Log Returns
    group["Log_Return"] = np.log(group["Return"] + 1) # remember that x = ln(1 + x), Taylor's polynomial on e^x

    # Forward 5-day return
    group["5_day_return"] = group["Log_Return"].rolling(5).sum().shift(-5)
    return group 

In [47]:
# Suppress FutureWarnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

# Calculate 5-day stock returns for the whole dataset from 1993 to 2000
crsp_data_1993_2000_R5 = crsp_data_1993_2000.groupby("PERMNO").apply(calculate_5_day_log_returns)

# Reset the index
crsp_data_1993_2000_R5 = crsp_data_1993_2000_R5.reset_index(drop = True)

# Display the result
crsp_data_1993_2000_R5

/var/folders/pl/z1lfm5xs7299k23shpwd6ngh0000gp/T/ipykernel_94651/1919957889.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  crsp_data_1993_2000_R5 = crsp_data_1993_2000.groupby("PERMNO").apply(calculate_5_day_log_returns)


,PERMNO,DlyCalDt,DlyVol,DlyClose,DlyLow,DlyHigh,DlyOpen,Return,Log_Return,5_day_return
0,10001,1993-01-04,150.0,14.5000,14.5000,14.500,14.5000,0.000000,0.000000,-5.310983e-02
1,10001,1993-01-07,228.0,14.5000,14.5000,14.500,14.5000,0.000000,0.000000,-1.739174e-02
2,10001,1993-01-08,1375.0,14.5000,14.2500,14.500,14.2500,0.000000,0.000000,-1.739174e-02
3,10001,1993-01-11,200.0,14.5000,14.5000,14.500,14.5000,-0.051724,-0.053110,1.387779e-17
4,10001,1993-01-13,700.0,13.7500,13.7500,14.000,14.0000,0.000000,0.000000,1.387779e-17
...,...,...,...,...,...,...,...,...,...,...
15555104,93316,2000-03-10,8555.0,15.2500,15.1250,15.250,15.1250,0.008197,0.008163,NaN
15555105,93316,2000-03-13,10685.0,15.3750,15.1250,15.375,15.2500,-0.020325,-0.020535,NaN
15555106,93316,2000-03-14,1800.0,15.0625,15.0625,15.375,15.3750,0.000000,0.000000,NaN
15555107,93316,2000-03-15,2100.0,15.0625,15.0625,15.375,15.0625,0.020747,0.020535,NaN


In [48]:
# Check the tail
filtered_data_10001 = crsp_data_1993_2000_R5[crsp_data_1993_2000_R5["PERMNO"] == 10001]
filtered_data_10001.tail(20)

,PERMNO,DlyCalDt,DlyVol,DlyClose,DlyLow,DlyHigh,DlyOpen,Return,Log_Return,5_day_return
1557,10001,2000-11-24,400.0,9.1250,9.12500,9.1250,9.1250,0.027397,0.027029,1.324523e-02
1558,10001,2000-11-27,802.0,9.3750,9.06250,9.3750,9.0625,-0.053333,-0.054808,6.805346e-02
1559,10001,2000-11-29,700.0,8.8750,8.87500,8.8750,8.8750,0.077465,0.074611,1.941809e-02
1560,10001,2000-11-30,100.0,9.5625,9.56250,9.5625,9.5625,-0.045752,-0.046831,6.624939e-02
1561,10001,2000-12-01,1000.0,9.1250,9.12500,9.5000,9.5000,-0.013699,-0.013793,8.004271e-02
1562,10001,2000-12-05,1000.0,9.0000,9.00000,9.0000,9.0000,0.055556,0.054067,2.597549e-02
1563,10001,2000-12-07,2160.0,9.5000,9.00000,9.5000,9.5000,0.000000,0.000000,2.597549e-02
1564,10001,2000-12-08,3865.0,9.5000,9.50000,9.5625,9.5000,0.026316,0.025975,-2.597549e-02
1565,10001,2000-12-13,1700.0,9.7500,9.75000,9.7500,9.7500,0.000000,0.000000,-7.979728e-17
1566,10001,2000-12-14,200.0,9.7500,9.75000,9.7500,9.7500,0.000000,0.000000,-7.979728e-17


In [49]:
# Check the head
filtered_data_10001.head(20)

,PERMNO,DlyCalDt,DlyVol,DlyClose,DlyLow,DlyHigh,DlyOpen,Return,Log_Return,5_day_return
0,10001,1993-01-04,150.0,14.50,14.50,14.50,14.50,0.000000,0.000000,-5.310983e-02
1,10001,1993-01-07,228.0,14.50,14.50,14.50,14.50,0.000000,0.000000,-1.739174e-02
2,10001,1993-01-08,1375.0,14.50,14.25,14.50,14.25,0.000000,0.000000,-1.739174e-02
3,10001,1993-01-11,200.0,14.50,14.50,14.50,14.50,-0.051724,-0.053110,1.387779e-17
4,10001,1993-01-13,700.0,13.75,13.75,14.00,14.00,0.000000,0.000000,1.387779e-17
5,10001,1993-01-14,400.0,13.75,13.75,13.75,13.75,0.000000,0.000000,1.387779e-17
6,10001,1993-01-19,200.0,13.75,13.75,13.75,13.75,0.036364,0.035718,-3.571808e-02
7,10001,1993-01-20,355.0,14.25,14.25,14.25,14.25,0.000000,0.000000,-3.571808e-02
8,10001,1993-01-21,300.0,14.25,14.25,14.25,14.25,-0.035088,-0.035718,3.571808e-02
9,10001,1993-01-22,200.0,13.75,13.75,13.75,13.75,0.000000,0.000000,1.387779e-17


In [50]:
# Fill NaN to 0
crsp_data_1993_2000_R5.fillna(0, inplace = True)

# Check the shape again
print(crsp_data_1993_2000_R5.shape) # (15555109, 10)

(15555109, 10)


In [51]:
# Drop Return and Log Return
crsp_data_1993_2000_R5 = crsp_data_1993_2000_R5.drop(["Return", "Log_Return"], axis = 1)

# Check shape again
crsp_data_1993_2000_R5.shape # (15555109, 8)

(15555109, 8)

In [ ]:
# We will have to select 70% images for training, and 30% images for validation
X_

In [33]:
# We will now create a folder named "chunk_folder"
# In this folder, we will store all the 20-day dataset

chunk_folder = "chunk_folder"

# Create a chunk_folder if it does not exist
if not os.path.exists(chunk_folder):
    os.makedirs(chunk_folder)

# I will have a log file that record any parts of the data that I skip
skip_log_file = os.path.join(chunk_folder, "skipped_chunk.txt")
crsp_data_ID = crsp_data_1993_2000_R5.groupby("PERMNO")

# Iterate through each group
for name, group in crsp_data_ID: # name here is the ID
    # Create a list to store output
    # This is the output in each image of the chunk
    y = []

    print(f"Current ID: {name}")
    # Divide into 20-day row chunks
    folder_path = os.path.join(chunk_folder, str(name))

    # Create a folder for each ID if the folder has not existed before
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # Go through each group, enumerate 20 rows, store in 1 csv
    for i, chunk in enumerate(range(0, len(group), 20)):
        chunk_data = group.iloc[chunk:chunk + 20]
        first_date = group.iloc[chunk]["DlyCalDt"]

        # Check if the chunk has exactly 20 rows
        if len(chunk_data) == 20:
            # We only want to take the last day return
            last_day_result = chunk_data["Ret_5d"].iloc[-1]

            # If they do not have the last day result, we skip that
            if not pd.isna(last_day_result):
                output = last_day_result 

                # Append the output
                y.append(output)

                file_path = os.path.join(folder_path, f"{first_date}.csv")
                chunk_data.to_csv(file_path, index = False)
            
            else:
                with open(skip_log_file, "a") as f:
                    f.write(f"Skipping incomplete chunk for PERMNO {name} and first date {first_date} (no 5-day later data)\n")
        
        else:
            with open(skip_log_file, "a") as f:
                f.write(f"Skipping incomplete chunk for PERMNO {name} and first date {first_date} (length: {len(chunk_data)})\n")
        
        # Export the y list to a CSV file
        output_file = os.path.join(folder_path, "output.csv")
        pd.Series(y).to_csv(output_file, index = False, header = ["output"])


Current ID: 10001
Current ID: 10002
Current ID: 10003
Current ID: 10009
Current ID: 10010
Current ID: 10011
Current ID: 10012
Current ID: 10016
Current ID: 10018
Current ID: 10019
Current ID: 10020
Current ID: 10021
Current ID: 10025
Current ID: 10026
Current ID: 10028
Current ID: 10032
Current ID: 10035
Current ID: 10037
Current ID: 10039
Current ID: 10042
Current ID: 10043
Current ID: 10044
Current ID: 10046
Current ID: 10047
Current ID: 10048
Current ID: 10051
Current ID: 10055
Current ID: 10056
Current ID: 10057
Current ID: 10059
Current ID: 10060
Current ID: 10062
Current ID: 10063
Current ID: 10064
Current ID: 10065
Current ID: 10066
Current ID: 10071
Current ID: 10074
Current ID: 10075
Current ID: 10077
Current ID: 10078
Current ID: 10079
Current ID: 10080
Current ID: 10082
Current ID: 10083
Current ID: 10084
Current ID: 10085
Current ID: 10086
Current ID: 10087
Current ID: 10089
Current ID: 10090
Current ID: 10092
Current ID: 10094
Current ID: 10095
Current ID: 10097
Current ID